In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

# Credits:
this notebook is based on the notbook ["Getting started with NLP for absolute beginners"](https://www.kaggle.com/code/jhoward/getting-started-with-nlp-for-absolute-beginners) by @Jeremy Howard

## Imports

In [ ]:
import tensorflow as tf
import tensorflow_hub as hub
import seaborn as sns
import matplotlib.pyplot as plt

# EDA

In [ ]:
train = pd.read_csv("/kaggle/input/us-patent-phrase-to-phrase-matching/train.csv")
print(train.shape)
train.head()

In [ ]:
train.describe(include='object')

In [ ]:
train.groupby('score').count()

## Score meanings according to [Data Description](https://www.kaggle.com/competitions/us-patent-phrase-to-phrase-matching/data?select=train.csv)

The scores are in the 0-1 range with increments of 0.25 with the following meanings:

    1.0 - Very close match. This is typically an exact match except possibly for differences in conjugation, quantity (e.g. singular vs. plural), and addition or removal of stopwords (e.g. “the”, “and”, “or”).
    0.75 - Close synonym, e.g. “mobile phone” vs. “cellphone”. This also includes abbreviations, e.g. "TCP" -> "transmission control protocol".
    0.5 - Synonyms which don’t have the same meaning (same function, same properties). This includes broad-narrow (hyponym) and narrow-broad (hypernym) matches.
    0.25 - Somewhat related, e.g. the two phrases are in the same high level domain but are not synonyms. This also includes antonyms.
    0.0 - Unrelated.


# Test Corpus

In [ ]:
test = pd.read_csv("/kaggle/input/us-patent-phrase-to-phrase-matching/test.csv")

In [ ]:
test.shape

In [ ]:
def cosine(u, v):
    """
    cosine similarity definition
    """
    return np.dot(u, v) / (np.linalg.norm(u) * np.linalg.norm(v))

# External Context

In [ ]:
# ajouter le contexte https://www.cooperativepatentclassification.org/Archive
# extraire le contexte du fichier xml

In [ ]:
titles = pd.read_csv("../input/cpc-codes/titles.csv")
print(titles.shape)
titles.head()

# HuggingFace AutoModelForSequenceClassification

In [ ]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import os
from scipy.stats import pearsonr
from scipy.stats import spearmanr # pour trouver les correspondances entre les résultats sur des échelles variées
from sklearn.model_selection import StratifiedGroupKFold

In [ ]:
import transformers
from transformers import TrainingArguments,Trainer
from transformers import AutoModelForSequenceClassification,AutoTokenizer

import torch
import datasets
from datasets import load_dataset, Dataset, DatasetDict

In [ ]:
import warnings
import logging
from IPython.display import display, HTML

# Corpora merge

In [ ]:
train['section'] = train["context"].str[0]
print(train.shape)
display(train.head())
titles=titles.rename(columns={"code": "context"})
display(titles.head())
train = train.merge(titles[["context", "title"]], on="context").rename(columns={"title": "context_title"})
train = train.merge(titles[["context", "title"]].rename(columns={"context": "section"}), on="section").rename(columns={"title": "section_title"})
display(train.head())

In [ ]:
logging.disable(logging.WARNING)
warnings.simplefilter('ignore')

# Deberta-v3-small

In [ ]:
#model_name = 'microsoft/deberta-v3-small' (online version)
model_name = '../input/debertav3small'

In [ ]:
tokenizer = AutoTokenizer.from_pretrained(model_name)

In [ ]:
tokenizer.all_special_tokens

In [ ]:
def clean(x):
    t = x.lower()
    t = t.replace("[",'')
    t = t.replace(";",'')
    t = t.replace(",",'')
    t = t.replace("]",'')
    t = t.replace(":",'')
    t = t.replace("(",'')
    t = t.replace(")",'')
    t = t.replace("{",'')
    t = t.replace("}",'')
    t = t.replace("/",' ')
    t = t.replace("-",' ')
    return t

In [ ]:
train["inputs"] = train["section_title"].apply(clean) + " [SEP] " + train["anchor"] + " [SEP] " + train["target"]

In [ ]:
def tok_func(x):
    return tokenizer(x["inputs"])

In [ ]:
train_ds = Dataset.from_pandas(train).rename_column('score', 'label')

In [ ]:
inps = "anchor","target","context","context_title","section_title"
tok_ds = train_ds.map(tok_func, batched=True, remove_columns=inps+('inputs','id','section'))

In [ ]:
tok_ds

In [ ]:
# bs - batch size
lr,bs = 8e-5,64
# wd: weight decay
wd,epochs = 0.01,4

In [ ]:
anchors = train["anchor"].unique()
np.random.shuffle(anchors)

In [ ]:
val_prop = 0.25
val_sz = int(len(anchors)*val_prop)
val_anchors = anchors[:val_sz]

In [ ]:
is_val = np.isin(train.anchor, val_anchors)
idxs = np.arange(len(train))
val_idxs = idxs[ is_val]
trn_idxs = idxs[~is_val]
len(val_idxs),len(trn_idxs)

In [ ]:
dds = DatasetDict({"train":tok_ds.select(trn_idxs),
             "test": tok_ds.select(val_idxs)})

In [ ]:
def corr(eval_pred): return {'pearson': np.corrcoef(*eval_pred)[0][1]}

In [ ]:
#args = TrainingArguments('outputs', learning_rate=lr, warmup_ratio=0.1, lr_scheduler_type='cosine', fp16=True,
    #evaluation_strategy="epoch", per_device_train_batch_size=bs, per_device_eval_batch_size=bs*2,
     #num_train_epochs=epochs, weight_decay=wd, report_to='none')
args = TrainingArguments('outputs', learning_rate=lr, warmup_ratio=0.1, lr_scheduler_type='cosine',
    evaluation_strategy="epoch", per_device_train_batch_size=bs, per_device_eval_batch_size=bs*2,
    num_train_epochs=epochs, weight_decay=wd, report_to='none')

In [ ]:
model = AutoModelForSequenceClassification.from_pretrained(model_name, num_labels=1)
trainer = Trainer(model, args, train_dataset=dds['train'], eval_dataset=dds['test'],
               tokenizer=tokenizer, compute_metrics=corr)

In [ ]:
dds['train'][0]

In [ ]:
trainer.train()

In [ ]:
trainer.save_model()

# Test corpus preprocessing

In [ ]:
test.head()

In [ ]:
test['section'] = test["context"].str[0]
print(test.shape)
display(titles.head())
test = test.merge(titles[["context", "title"]], on="context").rename(columns={"title": "context_title"})
test = test.merge(titles[["context", "title"]].rename(columns={"context": "section"}), on="section").rename(columns={"title": "section_title"})
display(test.head())

In [ ]:
test["inputs"] = test["section_title"].apply(clean) + " [SEP] " + test["anchor"] + " [SEP] " + test["target"]

In [ ]:
test_ds = Dataset.from_pandas(test)

In [ ]:
test_ds

In [ ]:
inps = "anchor","target","context","context_title","section_title"
test_tok_ds = test_ds.map(tok_func, batched=True, remove_columns=inps+('inputs','id','section'))

In [ ]:
preds = trainer.predict(test_tok_ds).predictions.astype(float)
preds

In [ ]:
type(preds)

In [ ]:
df_submission = pd.DataFrame()

In [ ]:
df_submission['id'] = test['id'].copy()

In [ ]:
df_submission

In [ ]:
preds_list = preds.tolist()

In [ ]:
preds_list

In [ ]:
flat_list = [x for xs in preds_list for x in xs]
flat_list

In [ ]:
df_submission['score' ] = flat_list

In [ ]:
df_submission

In [ ]:
#preds = np.clip(preds, 0, 1)
#preds

In [ ]:
#submission = datasets.Dataset.from_dict({
  #  'id': test_ds['id'],
 #   'score': preds
#})

df_submission.to_csv('submission.csv', index=False)